In [208]:
import pandas as pd
# from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import math

# Debugging a linear model

In this notebook, we go through an example linear classifier for predicting lung cancer from patient data. Here, we do some basic data loading and convert categorical variables to one hot encodings. 

In [192]:
df = pd.read_csv('processed.cleveland.data', header=None)
columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]
dtypes = ["num", "bin", "cat", "num", "num", "bin", "cat", "num", "bin", "num", "cat", "ord", "cat", "label"]
df.columns = columns


columns = columns[:11] + [columns[-1]]
dtypes = dtypes[:11] + [dtypes[-1]]
df = df[columns]


new_columns = []
for col,dt in zip(columns,dtypes): 
    if dt == 'cat': 
        dummies = pd.get_dummies(df[col], prefix=col)
        df = df.drop(columns=col).join(dummies)
        new_columns = new_columns + list(dummies.columns)
    else: 
        new_columns.append(col)
columns = new_columns
columns.remove('num')
df.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,num,cp_1.0,cp_2.0,cp_3.0,cp_4.0,restecg_0.0,restecg_1.0,restecg_2.0,slope_1.0,slope_2.0,slope_3.0
0,63.0,1.0,145.0,233.0,1.0,150.0,0.0,2.3,0,1,0,0,0,0,0,1,0,0,1
1,67.0,1.0,160.0,286.0,0.0,108.0,1.0,1.5,2,0,0,0,1,0,0,1,0,1,0
2,67.0,1.0,120.0,229.0,0.0,129.0,1.0,2.6,1,0,0,0,1,0,0,1,0,1,0
3,37.0,1.0,130.0,250.0,0.0,187.0,0.0,3.5,0,0,0,1,0,1,0,0,0,0,1
4,41.0,0.0,130.0,204.0,0.0,172.0,0.0,1.4,0,0,1,0,0,0,0,1,1,0,0


Next, we fit a linear model and check the accuracy -- the model has 84% accuracy. 

In [231]:
columns = df.drop(columns='num').columns
X,y = df.drop(columns='num').values, df['num'].values
y = (y > 0).astype(int)*2 - 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# get pvals using statsmodels
model = sm.OLS(y_train,X_train)
fii = model.fit()
pvals = fii.summary2().tables[1]['P>|t|']

# get model with sklearn
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

((y_pred>0) == (y_test>0)).mean()

0.8387096774193549

In [235]:
print(fii.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     14.70
Date:                Sun, 28 Aug 2022   Prob (F-statistic):           6.47e-27
Time:                        13:36:59   Log-Likelihood:                -299.77
No. Observations:                 272   AIC:                             631.5
Df Residuals:                     256   BIC:                             689.2
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0062      0.006      1.024      0.3

# Explaining the predictions
The linear model is not completely correct. Let's try to figure out why this is the case. First, let's explain the predictions of the model. For a linear model, we can directly do this by computing the contribution of each feature towards the prediction. We can also highlight the largest contributing features. 

### Variable meanings

| Variable | Definition | 
|:--- |:--- | 
| age | age in years |
| sex | sex (1 = male; 0 = female) |
| cp | cp: chest pain type |
| cp_1 | Value 1: typical angina |
| cp_2 | Value 2: atypical angina |
| cp_3 | Value 3: non-anginal pain |
| cp_4 | Value 4: asymptomatic |
| trestbps | esting blood pressure (in mm Hg on admission to the hospital) |
| chol | serum cholestoral in mg/dl | 
| fbs | (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false) | 
| restecg | resting electrocardiographic results | 
| restecg_0 | Value 0: normal |
| restecg_1 | Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) |
| restecg_2 | Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria |
| thalach | maximum heart rate achieved | 
| exang |  exercise induced angina (1 = yes; 0 = no) |
| oldpeak | ST depression induced by exercise relative to rest | 
| slope | the slope of the peak exercise ST segment | 
| slope_1 | upsloping | 
| slope_2 | flat | 
| slope_3 | downsloping | 

In [236]:
def explain(x): 
    xw = x*model.coef_
    for i,c in enumerate(columns): 
        print(f'{c}: {xw[i]:.2f}' + ('*' if abs(xw[i]) > 0.2 else '') + ('p' if pvals[i] < 0.05 else ''))
    print()
    b = model.intercept_
    y = b + xw.sum()
    print(f'(bias) {b:.2f} + (w^Tx): {xw.sum():.2f} = (total) {y:.2f}')
    print('No heart disease predicted' if y < 0 else 'Heart disease predicted')
    
explain(X_test[1])

age: 0.33*
sex: 0.49*p
trestbps: 0.55*
chol: 0.30*
fbs: 0.00
thalach: -0.44*
exang: 0.22*
oldpeak: 0.00p
cp_1.0: -0.00p
cp_2.0: -0.00p
cp_3.0: -0.00p
cp_4.0: 0.45*
restecg_0.0: -0.00p
restecg_1.0: -0.00
restecg_2.0: 0.04
slope_1.0: -0.00p
slope_2.0: 0.25*
slope_3.0: -0.00p

(bias) -1.63 + (w^Tx): 2.18 = (total) 0.56
Heart disease predicted


# Explain a misclassified example

So what about incorrect examples? 

In [234]:
for j in range(X_test.shape[0]):
    if (y_pred[j]>0) != (y_test[j]>0): 
        break
print(f'Ground truth: y_test={y_test[j]}')
explain(X_test[j])

Ground truth: y_test=1
age: 0.36*
sex: 0.49*p
trestbps: 0.50*
chol: 0.34*
fbs: 0.00
thalach: -0.64*
exang: 0.00
oldpeak: 0.02p
cp_1.0: -0.00p
cp_2.0: -0.00p
cp_3.0: -0.00p
cp_4.0: 0.45*
restecg_0.0: -0.03p
restecg_1.0: -0.00
restecg_2.0: 0.00
slope_1.0: -0.09p
slope_2.0: 0.00
slope_3.0: -0.00p

(bias) -1.63 + (w^Tx): 1.40 = (total) -0.23
No lung cancer predicted


Can you figure out why the model was incorrect? 